In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc
import pickle
import lightgbm as lgb
from sklearn.externals import joblib
import os

In [ ]:
os.makedirs('./result')
filename = './data/data.csv'
modelname1 = './model/lgb1.pkl'
modelname2 = './model/lgb2.pkl'
modelname3 = './model/lgb3.pkl'
modelname4 = './model/lgb4.pkl'
modelname5 = './model/lgb5.pkl'

In [ ]:
resultfile = './result/Prediction_Result.csv'
resultfileOrigin = './result/Prediction_Result_Origion.csv'
reader = pd.read_csv(filename, header=0, delimiter = '\t', index_col=0, iterator = True,)


In [ ]:
col_index_name = ['batch_no','age_app','sex_app','newvalue','seats','xb','quote_times','channel_source','nprem','si_tp','last_policy_complaint_cnt',]
cate_cols = ['sex_app','xb','channel_source','mark_price_int','mark_profit_int','is_lossend_nosms','is_high_risk_coordination']

In [5]:
lgbmodel1 = joblib.load(modelname1)
lgbmodel2 = joblib.load(modelname2)
lgbmodel3 = joblib.load(modelname3)
lgbmodel4 = joblib.load(modelname4)
lgbmodel5 = joblib.load(modelname5)

NameError: name 'modelname1' is not defined

In [ ]:
def predictionFunction(lgdmodel, test_data, col_index_name, currentTarget):
    res = pd.DataFrame() # 用于存放预测结果
    res['FRAMENUM'] = test_data['FRAMENUM']
    test_data = test_data[col_index_name]
    # TODO 分箱等
    # 1.将时间特征按照距今天多久处理
    import time
    import datetime
    for name in datetimeFeatures:
        test_data[name] = (pd.to_datetime(test_data[name]).dt.date - pd.datatime.now().date()).dt.days
        print(name)
        try:
            test_date[name] = (pd.to_datetime(test_data[name]).dt.date - pd.datetime.now().date()).dt.days
        except Exception:
            print("null date")
    # 2.处理类别型变量
    cate_cols = test_data.select_dtypes(include='object').columns
    cate_cols = list(cate_cols)
    print(cate_cols)
    # 将object类型转成category类型
    for c in cate_cols:
        test_data[c] = test_data[c].astype('category')
    pred_list_prob = lgbmodel.predict_proba(test_data,)
    res[currentTarget] = pred_list_prob[:,1]
    return res

In [ ]:
chunkSize = 1000000
predict_result = pd.DataFrame()
loop = True
i = 1
while loop:
    try:
        chunk  = reader.get_chunk(chunkSize)
        newdata = chunk
        newdata['FRAMENUM'] = newdata.index
        print("predicting {} test_data".format(i))
        predict_part1 = predictFunction(lgbmodel1, newdata, col_index_name, 'QUALITY')
        predict_part2 = predictFunction(lgbmodel2, newdata, col_index_name, 'ATTITUDE')
        predict_part3 = predictFunction(lgbmodel3, newdata, col_index_name, 'TIME')
        predict_part4 = predictFunction(lgbmodel4, newdata, col_index_name, 'SKILL')
        predict_part5 = predictFunction(lgbmodel5, newdata, col_index_name, 'MONEY')
        predict_part = pd.concat([predict_part1['FRAMENUM'],predict_part1['QUALITY'],predict_part2['ATTITUDE'],
                                  predict_part3['TIME'],predict_part4['SKILL'],predict_part5['MONEY']], axis=1)
        print("predict {} test_data is done".format(i))
        # 拼接预测结果
        predict_result = predict_result.append(predict_part)
        # 输出原始预测结果
        predict_part.to_csv(resultfileOrigin, mode = 'a', index=False, sep=',')
        print("to_csv {} test_data is done".format(i))
        i = i + 1
    except StopIteration:
        loop = False
        print("Iteration is stopped")



In [ ]:
# 找到最大的敏感类型的预测概率值和对应的敏感类型英文名
predict_result['PREDICTION_VALUE'] = predict_result[['QUALITY','ATTITUDE','TIME','SKILL''MONEY']].astype(float).idmax(axis=1)
predict_result['RISK_TYPE'] = predict_result[['QUALITY','ATTITUDE','TIME','SKILL''MONEY']].astype(float).idxmax(axis = 1)
# 取出前千分之五
count = int(len(predict_result) * 0.005)
c = predict_result.sort_values(by="prediction_value", ascending = False).head(count)
import time
prediction_final = pd.DataFrame()
prediction_final['FRAMENUM'] = prediction_desceding['FRAMENUM']
prediction_final['RISK_TYPE'] = prediction_desceding['RISK_TYPE']
prediction_final['PREDICTION_VALUE'] = prediction_desceding['PREDICTION_VALUE']
prediction_final['INSERT_TIME'] = time.strftime("%Y-%m-%d", time.localtime(time.time()))
prediction_final = prediction_final.drop_duplicates(subset = ['FRAMENUM', 'RISK_TYPE'])

print(prediction_final.head())

prediction_final.to_csv(resultfile, index=False, header = 0, sep=",")